# Part 2: Deep Dive - 문헌 검색 (Agent Laboratory / PaSa)

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. Agent Laboratory와 PaSa의 코드 구조 이해
2. arXiv/Scholar 자동 검색 실행
3. 검색 로직 커스텀 (쿼리, 필터, 정렬)

---
## Setup

In [ ]:
# Cell 1: Google Drive Mount
from google.colab import drive
drive.mount('/content/drive/')

import os
WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
os.makedirs(WORKSHOP_DIR, exist_ok=True)
os.chdir(WORKSHOP_DIR)
print(f"작업 폴더: {os.getcwd()}")

In [ ]:
# Cell 2: 패키지 설치 + 저장소 클론
!pip install arxiv openai python-dotenv -q

# Agent Laboratory 클론
!git clone https://github.com/SamuelSchmidgall/AgentLaboratory.git 2>/dev/null || echo "Already cloned"

# PaSa 클론  
!git clone https://github.com/bytedance/pasa.git 2>/dev/null || echo "Already cloned"

print("설치 완료!")

In [ ]:
# Cell 3: API Key 로딩
import os
from dotenv import load_dotenv

load_dotenv()

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
print("API Key 설정 완료!" if OPENAI_API_KEY else "API Key 없음 - 일부 기능 제한")

---
## 1. 파악하기: Agent Laboratory 코드 구조

### Agent Laboratory 구조

```
AgentLaboratory/
├── agents/
│   ├── literature_agent.py   # 문헌 검색 에이전트 ⭐
│   ├── experiment_agent.py   # 실험 에이전트
│   └── report_agent.py       # 보고서 에이전트
├── tools/
│   └── arxiv_search.py       # arXiv API 래퍼 ⭐
└── run.py                    # 메인 실행
```

**핵심 흐름:**
1. 연구 주제 입력
2. `literature_agent`가 arXiv에서 관련 논문 검색
3. 논문 요약 + 실험 제안
4. 보고서 생성

In [ ]:
# Cell 6: Agent Laboratory 핵심 모듈 확인
import os

# 폴더 구조 확인
agent_lab_path = "AgentLaboratory"
if os.path.exists(agent_lab_path):
    print("📁 AgentLaboratory 구조:")
    for root, dirs, files in os.walk(agent_lab_path):
        level = root.replace(agent_lab_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # 처음 5개 파일만
            if file.endswith('.py'):
                print(f"{subindent}{file}")
else:
    print("AgentLaboratory가 클론되지 않았습니다. Cell 2를 실행하세요.")

In [ ]:
# Cell 7: arXiv 검색 모듈 직접 살펴보기
import arxiv

# arXiv 검색 함수 (Agent Laboratory 스타일)
def search_arxiv_papers(query, max_results=10, sort_by="relevance"):
    """
    Agent Laboratory의 arxiv_search.py 핵심 로직
    """
    sort_criterion = {
        "relevance": arxiv.SortCriterion.Relevance,
        "date": arxiv.SortCriterion.SubmittedDate,
    }.get(sort_by, arxiv.SortCriterion.Relevance)
    
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=sort_criterion
    )
    
    papers = []
    for result in search.results():
        papers.append({
            "title": result.title,
            "authors": [a.name for a in result.authors],
            "abstract": result.summary,
            "published": result.published.strftime("%Y-%m-%d"),
            "pdf_url": result.pdf_url,
            "arxiv_id": result.entry_id.split('/')[-1]
        })
    return papers

print("search_arxiv_papers 함수 정의 완료")
print(f"시그니처: search_arxiv_papers(query, max_results=10, sort_by='relevance')")

### PaSa (Paper Search Agent) 구조

```
pasa/
├── agents/
│   ├── crawler.py     # 논문 크롤링 에이전트 ⭐
│   └── selector.py    # 관련성 평가 (PPO 학습됨) ⭐
├── search/
│   ├── arxiv.py       # arXiv 검색
│   └── scholar.py     # Google Scholar 검색
└── run_search.py
```

**핵심 차이점:**
- PaSa는 **PPO로 학습된 Selector**가 관련성 평가
- Agent Laboratory보다 더 정교한 필터링

### 질문

두 도구의 구조를 비교해보세요:
- Agent Laboratory: 문헌 → 실험 → 보고서 (E2E 파이프라인)
- PaSa: 검색 특화 (PPO 기반 선택)

본인 연구에는 어떤 접근이 더 맞을까요?

---
## 2. 써보기: arXiv 논문 검색

### 샘플 데이터

테스트할 연구 주제:
- "LLM reasoning and cognitive psychology"
- "AI agent for scientific discovery"

In [ ]:
# Cell 10: 샘플 검색 실행 (발표자 데모)
sample_query = "LLM reasoning cognitive psychology"

print(f"🔍 검색어: {sample_query}\n")
results = search_arxiv_papers(sample_query, max_results=5)

print(f"📚 검색 결과: {len(results)}개 논문\n")
for i, paper in enumerate(results, 1):
    print(f"{i}. {paper['title']}")
    print(f"   저자: {', '.join(paper['authors'][:3])}...")
    print(f"   날짜: {paper['published']}")
    print(f"   ID: {paper['arxiv_id']}")
    print()

In [ ]:
# Cell 11: 결과 시각화
import pandas as pd

# DataFrame으로 변환
df = pd.DataFrame(results)
df['abstract_preview'] = df['abstract'].str[:100] + '...'
df['author_count'] = df['authors'].apply(len)

print("📊 검색 결과 요약")
display(df[['title', 'published', 'author_count']].head())

### DIY: 본인 연구 주제로 검색해보세요

In [ ]:
# DIY Cell 13: 본인 연구 주제 검색
# 힌트: my_query 변수만 바꾸면 됩니다

my_query = "your research topic here"  # <- 이 부분 수정!

my_results = search_arxiv_papers(my_query, max_results=10)

print(f"🔍 검색어: {my_query}")
print(f"📚 결과: {len(my_results)}개 논문\n")

for i, paper in enumerate(my_results, 1):
    print(f"{i}. {paper['title']}")
    print(f"   {paper['published']} | {paper['arxiv_id']}")
    print()

In [ ]:
# DIY Cell 14: 검색 결과 분석
# 질문: 검색 결과 중 관련성 높은 논문은 몇 개인가요?

# 관련성 평가 (1-5점)
# 메모:
# 논문 1: 
# 논문 2:
# 논문 3:

---
## 3. 바꿔보기: 검색 로직 커스텀

### 수정 포인트

| 항목 | 기본값 | 커스텀 아이디어 |
|------|--------|----------------|
| 검색 필드 | 전체 | 제목만, 초록만 |
| 카테고리 | 전체 | cs.CL, cs.AI, q-bio |
| 날짜 범위 | 전체 | 최근 2년만 |
| 정렬 | relevance | 최신순 |

In [ ]:
# Cell 16: 커스텀 검색 함수 (Before/After)

# Before: 기본 검색
def search_basic(query):
    return search_arxiv_papers(query, max_results=10)

# After: 심리학 특화 검색
def search_psychology_papers(query, max_results=10, years_back=2):
    """
    심리학 연구용 커스텀 검색
    - q-bio.NC (Neurons and Cognition) 카테고리 우선
    - 최근 N년 논문만
    - 제목+초록에서 검색
    """
    from datetime import datetime, timedelta
    
    # 날짜 필터 계산
    cutoff_date = datetime.now() - timedelta(days=years_back*365)
    
    # arXiv 쿼리 문법: cat:카테고리 AND (ti:제목 OR abs:초록)
    enhanced_query = f"(ti:{query} OR abs:{query})"
    
    search = arxiv.Search(
        query=enhanced_query,
        max_results=max_results * 2,  # 필터링 위해 더 많이 검색
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    papers = []
    for result in search.results():
        # 날짜 필터
        if result.published.replace(tzinfo=None) < cutoff_date:
            continue
        
        papers.append({
            "title": result.title,
            "authors": [a.name for a in result.authors],
            "abstract": result.summary,
            "published": result.published.strftime("%Y-%m-%d"),
            "categories": result.categories,
            "pdf_url": result.pdf_url
        })
        
        if len(papers) >= max_results:
            break
    
    return papers

# 비교 실행
query = "cognitive load working memory"
print("=== 기본 검색 ===")
basic = search_basic(query)
print(f"결과: {len(basic)}개")

print("\n=== 심리학 특화 검색 (최근 2년) ===")
psych = search_psychology_papers(query, years_back=2)
print(f"결과: {len(psych)}개")

In [ ]:
# DIY Cell 17: 본인만의 필터 추가하기

def my_custom_search(query, max_results=10):
    """
    TODO: 본인 연구에 맞게 커스텀하세요
    
    아이디어:
    - 특정 저자 필터: if 'Author Name' in authors
    - 키워드 필터: if 'keyword' in abstract.lower()
    - 카테고리 필터: if 'cs.CL' in categories
    """
    # 여기에 코드 작성
    pass

# 테스트
# results = my_custom_search("your query")

In [ ]:
# Cell 18: LLM으로 검색 쿼리 자동 생성
from openai import OpenAI

def generate_search_queries(research_topic, num_queries=3):
    """
    연구 주제에서 다양한 검색 쿼리 자동 생성
    """
    client = OpenAI()
    
    prompt = f"""Generate {num_queries} different arXiv search queries for the following research topic.
Each query should focus on a different aspect of the topic.
Return only the queries, one per line.

Research topic: {research_topic}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    queries = response.choices[0].message.content.strip().split('\n')
    return [q.strip() for q in queries if q.strip()]

# 테스트 (API key 있을 때만)
if OPENAI_API_KEY:
    topic = "AI agents for psychological research"
    queries = generate_search_queries(topic)
    print(f"생성된 쿼리:")
    for i, q in enumerate(queries, 1):
        print(f"{i}. {q}")
else:
    print("API key 없음 - 이 기능은 건너뜁니다")

In [ ]:
# DIY Cell 19: 자동 생성된 쿼리로 검색

my_topic = "your research topic"  # <- 수정

if OPENAI_API_KEY:
    # 쿼리 생성
    auto_queries = generate_search_queries(my_topic, num_queries=3)
    
    # 각 쿼리로 검색
    all_results = {}
    for query in auto_queries:
        results = search_arxiv_papers(query, max_results=3)
        all_results[query] = results
        print(f"\n🔍 '{query}' → {len(results)}개")
        for p in results:
            print(f"   - {p['title'][:60]}...")

In [ ]:
# Cell 20: 검색 결과 비교 (원본 vs 수정본)
query = "cognitive psychology LLM"

# 기본 검색
basic_results = search_arxiv_papers(query, max_results=5)

# 심리학 특화 검색
psych_results = search_psychology_papers(query, max_results=5, years_back=2)

print("📊 검색 방법 비교\n")
print(f"기본 검색: {len(basic_results)}개")
for p in basic_results[:3]:
    print(f"  - {p['title'][:50]}... ({p['published']})")

print(f"\n심리학 특화 (2년): {len(psych_results)}개")
for p in psych_results[:3]:
    print(f"  - {p['title'][:50]}... ({p['published']})")

---
## 4. 토론

### 토론 질문

1. **검색 결과의 질을 어떻게 평가할 수 있을까?**
   - precision vs recall
   - 관련성 점수?

2. **Agent Laboratory vs PaSa 중 어떤 것을 베이스로 쓸까?**
   - Agent Lab: E2E 파이프라인 (문헌→실험→보고서)
   - PaSa: 검색 특화 (PPO 기반 선택)

3. **추가하고 싶은 기능은?**
   - PubMed 통합?
   - 한글 논문 (DBpia, RISS)?
   - 자동 PDF 다운로드?

### 다음 단계

이어서 실습할 내용:
- **Part 3**: LitLLM으로 검색된 논문을 바탕으로 Related Work 초안 생성
- **Part 4**: AgentReview로 작성한 논문에 피드백 받기

### 과제 아이디어
```python
# PubMed API 모듈 추가하기
def search_pubmed(query):
    # E-utilities API 사용
    pass

# 여러 DB 결과 통합하기
def search_all(query):
    arxiv = search_arxiv_papers(query)
    pubmed = search_pubmed(query)
    return merge_and_dedupe(arxiv, pubmed)
```